In [1]:
import socket
import rawpy
import time
import numpy as np
from PIL import Image
import hmac



# Divide the image into into 920 bit chunks
def divide_image_into_pixel_groups(image_path, group_pixel_count=30):
    # Step 1: Loading the Image
    with Image.open(image_path).convert('RGB') as img:
        width, height = img.size
        

        # Step 2: Calculating Group Dimensions
        # Find a suitable width and height for the groups that would approximately contain 1024 pixels
        group_width = int(np.sqrt(group_pixel_count))
        group_height = group_pixel_count // group_width

        # Adjust group_height in case group_width * group_height < 1024
        if group_width * group_height < group_pixel_count:
            group_height += 1

        groups = []

        # Step 3: Dividing the Image
        for y in range(0, height, group_height):
            for x in range(0, width, group_width):
                # Define the bounding box for each group
                box = (x, y, x + group_width, y + group_height)
                # Crop the image to the box to get the group
                group = img.crop(box)
                groups.append(group)

        return groups


def send_bmp_image(file_path, ip, port, p):
    groups = divide_image_into_pixel_groups(file_path)
    

    # Create a UDP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    i =0
    try:
        for pixel_group in groups:
  
            chunk = pixel_group.tobytes()
            # Send chunk index and chunk data
            if np.random.rand() < p:
                mac = hmac.new(b'key', chunk, 'sha384').digest()
                sock.sendto(i.to_bytes(4, 'big') + chunk + mac , (ip, port))
                time.sleep(0.0001)
            else:
                print("modify, sequence number = ", i)
                mac = hmac.new(b'key', b'modified', 'sha384').digest()
                if np.random.rand() < .5:
                    sock.sendto(i.to_bytes(4, 'big') + chunk + mac, (ip, port))
                time.sleep(0.0001)
            if i % 10000 == 0:
                print("Sent chunk ", i, " of ", len(groups))
            i += 1
            

        # Send a final message indicating completion
        sock.sendto(b'END', (ip, port))
    finally:
        print("Image sent")
        sock.close()
    return i

# Example usage
file_path = 'output.png'
destination_ip = '0.0.0.0'
destination_port = 23423


# packet success probability
p = .9
send_bmp_image(file_path, destination_ip, destination_port, p = p)


Sent chunk  0  of  78012
modify, sequence number =  7
modify, sequence number =  16
modify, sequence number =  39
modify, sequence number =  43
modify, sequence number =  58
modify, sequence number =  81
modify, sequence number =  84
modify, sequence number =  91
modify, sequence number =  95
modify, sequence number =  115
modify, sequence number =  123
modify, sequence number =  144
modify, sequence number =  172
modify, sequence number =  176
modify, sequence number =  178
modify, sequence number =  196
modify, sequence number =  198
modify, sequence number =  215
modify, sequence number =  222
modify, sequence number =  227
modify, sequence number =  233
modify, sequence number =  244
modify, sequence number =  250
modify, sequence number =  251
modify, sequence number =  255
modify, sequence number =  261
modify, sequence number =  262
modify, sequence number =  281
modify, sequence number =  302
modify, sequence number =  309
modify, sequence number =  312
modify, sequence number 

78012

In [2]:
groups = divide_image_into_pixel_groups("output.png", group_pixel_count=30)
# combine the groups into a single image

len(groups)




78012

In [3]:
# groups[0] = Image.fromarray(np.zeros(np.array(groups[1]).shape, dtype=np.uint8))
# groups[0]
np.array(groups[1]).shape

(6, 5, 3)

In [4]:
import hmac

len(hmac.new(b'key', b'message', 'sha384').digest())


48